# Advent of Code 2024, Day 13

In [ ]:
with open('input.txt', 'r') as f:
    puzzle_input = f.read().strip()

In [ ]:
# puzzle_input = """Button A: X+94, Y+34
# Button B: X+22, Y+67
# Prize: X=8400, Y=5400
# 
# Button A: X+26, Y+66
# Button B: X+67, Y+21
# Prize: X=12748, Y=12176
# 
# Button A: X+17, Y+86
# Button B: X+84, Y+37
# Prize: X=7870, Y=6450
# 
# Button A: X+69, Y+23
# Button B: X+27, Y+71
# Prize: X=18641, Y=10279"""

## [First Puzzle:](https://adventofcode.com/2024/day/13)

In [ ]:
lines = puzzle_input.split('\n')
values = [split[1] for line in lines if (split := line.split(':')) and len(split) > 1]
button_a_values = [values[i] for i in range(0, len(values), 3)]
button_b_values = [values[i] for i in range(1, len(values), 3)]
prize_values = [values[i] for i in range(2, len(values), 3)]

In [ ]:
import re

for i in range(0, len(prize_values)):
    prize_values[i] = re.findall(r'\b\d+\b', prize_values[i])
    prize_values[i] = [int(value) for value in prize_values[i]]

    button_a_values[i] = re.findall(r'\b\d+\b', button_a_values[i])
    button_a_values[i] = [int(value) for value in button_a_values[i]]

    button_b_values[i] = re.findall(r'\b\d+\b', button_b_values[i])
    button_b_values[i] = [int(value) for value in button_b_values[i]]

In [ ]:
def find_solutions(x1, y1, x2, y2, x, y, a_range, b_range):
    solutions = []
    for a in a_range:
        for b in b_range:
            if a * x1 + b * x2 == x and a * y1 + b * y2 == y:
                solutions.append((a, b))
    return solutions

In [ ]:
solutionses = [
    find_solutions(*button_a_values[i], *button_b_values[i], *prize_values[i], a_range=range(0, 100),
                   b_range=range(0, 100)) for i in range(len(prize_values))
]

In [ ]:
solutionses_with_costs = [[(solution[0], solution[1], 3 * solution[0] + solution[1]) for solution in solutions] for
                       solutions in solutionses]

In [ ]:
sum(
    min(solutions, key=lambda x: x[2])[2] if solutions else 0
    for solutions in solutionses_with_costs
)

## [Second Puzzle:](https://adventofcode.com/2024/day/13/#part2)

In [ ]:
lines = puzzle_input.split('\n')
values = [split[1] for line in lines if (split := line.split(':')) and len(split) > 1]
button_a_values = [values[i] for i in range(0, len(values), 3)]
button_b_values = [values[i] for i in range(1, len(values), 3)]
prize_values = [values[i] for i in range(2, len(values), 3)]

In [ ]:
import re

for i in range(0, len(prize_values)):
    prize_values[i] = re.findall(r'\b\d+\b', prize_values[i])
    prize_values[i] = [int(value) for value in prize_values[i]]

    button_a_values[i] = re.findall(r'\b\d+\b', button_a_values[i])
    button_a_values[i] = [int(value) for value in button_a_values[i]]

    button_b_values[i] = re.findall(r'\b\d+\b', button_b_values[i])
    button_b_values[i] = [int(value) for value in button_b_values[i]]

In [ ]:
BIG_NUMBER = 10000000000000

In [ ]:
for i in range(0, len(prize_values)):
    prize_values[i] = [value + BIG_NUMBER for value in prize_values[i]]

In [ ]:
def extended_gcd(a, b):
    """
    Returns (g, x, y) such that a*x + b*y = g = gcd(a, b).
    """
    if b == 0:
        return a, 1, 0
    g, x1, y1 = extended_gcd(b, a % b)
    # By back-substitution:
    # a*x + b*y = g
    # => (b)*(x1) + (a % b)*(y1) = g
    return g, y1, x1 - (a // b) * y1

def smallest_sum_solution(x1, y1, x2, y2, x0, y0):
    """
    Returns the (a, b) that solves:
        a*x1 + b*x2 = x0
        a*y1 + b*y2 = y0
    and that minimizes a + b.
    
    If no such solution exists, returns None.
    """

    # --- 1) Solve first equation x1*a + x2*b = x0 via EEA.
    g1, s1, t1 = extended_gcd(x1, x2)  # gcd(x1,x2) = g1
    if x0 % g1 != 0:
        return None  # No solution possible for eqn1
    
    # Particular solution to eqn1:
    #  x1*(s1) + x2*(t1) = g1
    # So for x0, we scale up by factor:
    scale = x0 // g1
    a_part = s1 * scale
    b_part = t1 * scale
    
    # General solution: a = a_part + k*(x2/g1), b = b_part - k*(x1/g1)
    dx = x2 // g1
    dy = x1 // g1

    # --- 2) Impose second equation: y1*a + y2*b = y0
    # M = y1*(x2/g1) - y2*(x1/g1), C = y0 - (y1*a_part + y2*b_part)
    M = y1*dx - y2*dy
    C = y0 - (y1*a_part + y2*b_part)

    def a_of(k):
        return a_part + k*dx
    def b_of(k):
        return b_part - k*dy

    if M == 0:
        # 2a) If M=0 but C != 0 => no solutions
        if C != 0:
            return None
        
        # 2b) If M=0 and C=0 => infinite solutions
        # => second eq is automatically satisfied for *all* integer k.
        # We must minimize a(k)+b(k) subject to a(k) >= 0, b(k) >= 0 (if required).
        
        # sum_of(k) = a_part + b_part + k*(dx - dy)
        Delta = dx - dy
        
        # Next, we find the integer range for k from a>=0 and b>=0:
        #   a(k) = a_part + k*dx >= 0
        #   b(k) = b_part - k*dy >= 0
        #
        # We'll solve these inequalities to get a range for k.  

        # We'll define a helper that returns the valid integer range for k from each inequality:
        def range_from_inequality(base, step):
            # base + k*step >= 0
            # => k*step >= -base
            # => k >= -base/step  (if step>0)
            # or k <= -base/step  (if step<0)
            
            # We'll return a half-open interval [low, high] or something similar, possibly infinite.
            if step == 0:
                if base < 0:
                    # Then there's no solution
                    return None  # means no valid k
                else:
                    # base>=0 => no restriction from this inequality
                    return -float('inf'), float('inf')
            else:
                boundary = -base / step
                if step > 0:
                    # k >= boundary
                    return boundary, float('inf')
                else:
                    # k <= boundary
                    return -float('inf'), boundary

        rng_a = range_from_inequality(a_part, dx)
        rng_b = range_from_inequality(b_part, -dy)
        if rng_a is None or rng_b is None:
            return None  # no valid k at all
        # Intersection of these intervals:
        low_k = max(rng_a[0], rng_b[0])
        high_k = min(rng_a[1], rng_b[1])

        # Because we want k to be an integer:
        # low_k_int = smallest integer >= low_k
        # high_k_int = largest integer <= high_k
        import math
        low_k_int = math.ceil(low_k)
        high_k_int = math.floor(high_k)

        if low_k_int > high_k_int:
            return None  # no integer in that intersection => no solution

        # Now sum_of(k) = (a_part + b_part) + k * Delta. 
        # If Delta>0 => sum_of(k) is minimized when k is as small as possible. 
        # If Delta<0 => sum_of(k) is minimized when k is as large as possible. 
        if Delta > 0:
            best_k = low_k_int
        elif Delta < 0:
            best_k = high_k_int
        else:
            # Delta=0 => sum_of(k) is constant => pick any valid k
            best_k = low_k_int  # or anything in [low_k_int, high_k_int]
        
        A = a_of(best_k)
        B = b_of(best_k)
        return (A, B) if (A >= 0 and B >= 0) else None

    else:
        # M != 0 => we have exactly one integer k if C % M == 0
        if C % M != 0:
            return None
        k_fixed = C // M
        A = a_of(k_fixed)
        B = b_of(k_fixed)

        # We require nonnegative solutions, check them
        if A < 0 or B < 0:
            return None
        
        # That's the only solution, so it automatically has minimal a+b among solutions.
        return A, B

In [ ]:
smallest_sum_solutions = [
    smallest_sum_solution(*button_a_values[i], *button_b_values[i], *prize_values[i])
    for i in range(len(prize_values))
]

In [ ]:
solutions_with_costs = [(solution[0], solution[1], 3 * solution[0] + solution[1]) for solution in smallest_sum_solutions if solution]

In [ ]:
sum(
    solution[2] if solution else 0
    for solution in solutions_with_costs
)